<a href="https://colab.research.google.com/github/roshantac/EVA4/blob/master/Assignment15/B/finlworkinTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!nvidia-smi

['Wed May 20 02:04:53 2020       ',
 '+-----------------------------------------------------------------------------+',
 '| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |',
 '|-------------------------------+----------------------+----------------------+',
 '| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |',
 '| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |',
 '|===============================+======================+======================|',
 '|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |',
 '| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |',
 '+-------------------------------+----------------------+----------------------+',
 '                                                                               ',
 '+-----------------------------------------------------------------------------+',
 '| Processes:                          

In [2]:
!git clone https://github.com/roshantac/Unet.git

Cloning into 'Unet'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 51 (delta 20), reused 30 (delta 9), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [0]:
!mkdir Unet/data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir Dataset
!cp '/content/drive/My Drive/Rohit dataset/Dataset.zip' Dataset
#!cp '/content/drive/My Drive/sample10.zip' temp

In [0]:
destDataDir = '/content/temp/'
path_to_zip_file = '/content/temp/Dataset.zip'
import zipfile
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(destDataDir)

In [0]:
# destDataDir = '/content/temp/'
# path_to_zip_file = '/content/temp/sample10.zip'
# import zipfile
# with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
#     zip_ref.extractall(destDataDir)

In [0]:
# !mv '/content/temp/sample10k/depth' '/content/Unet/data/'
# !mv '/content/temp/sample10k/fgbg' '/content/Unet/data/'
# !mv '/content/temp/sample10k/mask' '/content/Unet/data/'

In [0]:
# !rm -rf temp

In [1]:
%cd Unet/

/content/Unet


In [0]:
import argparse
import logging
import os
import sys

import numpy as np
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm

from eval import eval_net
from unet import UNet

from torch.utils.tensorboard import SummaryWriter
from utils.dataset import BasicDataset
from torch.utils.data import DataLoader, random_split

In [0]:
root = '/content/'
dir_checkpoint = 'checkpoints/'

def train_net(net, device, epochs = 10, batch_size = 1, lr = 0.001, val_percent  =0.2, save_cp = True, img_scale = 0.5):
    dataset = BasicDataset(root)
    n_val = int(len(dataset) * val_percent)
    n_train = len(dataset) - n_val
    train, val = random_split(dataset, [n_train, n_val])
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True, drop_last=True)

    writer = SummaryWriter(comment=f'LR_{lr}_BS_{batch_size}_SCALE_{img_scale}')
    global_step = 0

    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_cp}
        Device:          {device.type}
        Images scaling:  {img_scale}
    ''')

    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min' if net.n_classes > 1 else 'max', patience=2)
    if net.n_classes > 1:
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.BCEWithLogitsLoss()

    for epoch in range(epochs):
        net.train()

        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                batch['bg'] = batch['bg'].to(device, dtype = torch.float32)
                batch['fgbg'] = batch['fgbg'].to(device, dtype = torch.float32)
                batch['mask'] = batch['mask'].to(device, dtype = torch.float32)
                batch['depth'] = batch['depth'].to(device, dtype = torch.float32) 
                # assert imgs.batch[1] == net.n_channels, \
                #     f'Network has been defined with {net.n_channels} input channels, ' \
                #     f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                #     'the images are loaded correctly.'
                # bg = bg.to(device=device, dtype = torch.float32)
                # imgs = imgs.to(device=device, dtype=torch.float32)
                # mask_type = torch.float32 if net.n_classes == 1 else torch.long
                # true_masks = true_masks.to(device=device, dtype=mask_type)
                # true_depth = true_depth.to(device=device, dtype=torch.float32) #------------------------add----------------

                depth_pred, masks_pred = net(batch)
                loss1 = criterion(masks_pred, batch['mask'])
                loss2 = criterion(depth_pred, batch['depth']) #------------------------add----------------
                loss = loss1 + loss2 #------------------------add----------------
                epoch_loss += loss.item()
                writer.add_scalar('Loss/train', loss.item(), global_step)

                pbar.set_postfix(**{'loss (batch)': loss.item()})

                optimizer.zero_grad()
                loss.backward()
                nn.utils.clip_grad_value_(net.parameters(), 0.1)
                optimizer.step()

                pbar.update(batch['fgbg'].shape[0])
                global_step += 1
                # if global_step % (len(dataset) // (10 * batch_size)) == 0:
                #     for tag, value in net.named_parameters():
                #         tag = tag.replace('.', '/')
                #         writer.add_histogram('weights/' + tag, value.data.cpu().numpy(), global_step)
                #         writer.add_histogram('grads/' + tag, value.grad.data.cpu().numpy(), global_step)
                #     val_score = eval_net(net, val_loader, device)
                #     scheduler.step(val_score)
                #     writer.add_scalar('learning_rate', optimizer.param_groups[0]['lr'], global_step)

                #     if net.n_classes > 1:
                #         logging.info('Validation cross entropy: {}'.format(val_score))
                #         writer.add_scalar('Loss/test', val_score, global_step)
                #     else:
                #         logging.info('Validation Dice Coeff: {}'.format(val_score))
                #         writer.add_scalar('Dice/test', val_score, global_step)

                #     writer.add_images('images', imgs, global_step)
                #     if net.n_classes == 1:
                #         writer.add_images('masks/true', true_masks, global_step)
                #         writer.add_images('masks/pred', torch.sigmoid(masks_pred) > 0.5, global_step)

        if save_cp:
            try:
                os.mkdir(dir_checkpoint)
                logging.info('Created checkpoint directory')
            except OSError:
                pass
            torch.save(net.state_dict(),
                       dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
            logging.info(f'Checkpoint {epoch + 1} saved !')

    writer.close()



In [0]:
sample = next(iter(train_dl))
[(k, v.shape) for k, v in sample.items()]

In [4]:
net = UNet(n_channels=6, n_classes=1, bilinear=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device=device)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.

In [0]:
def show(tensors, figsize= (10,10), *args, **kwargs):
  try:
    tensors = tensors.detach().cpu()
  except:
    pass
  grid_tensor = torchvision.utils.make_grid(tensors, *args, **kwargs)  
  grid_image  = grid_tensor.permute(1, 2, 0)
  plt.figure(figsize = figsize)
  plt.imshow(grid_image)
  plt.xticks([])  
  plt.yticks([])
  plt.show()

In [0]:
def show_pred(tensors, *args, **kwargs):
  tensors = (tensors * std[None, : , None, None]) + mean[None, :, None, None]
  show(tensors, *args, **kwargs)

In [0]:
show(imgs, nrow=8)

In [9]:
from torchsummary import summary

summary(net, (3,224,224))

TypeError: ignored

In [7]:

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

logging.info(f'Using device {device}')

# Change here to adapt to your data
# n_channels=3 for RGB images
# n_classes is the number of probabilities you want to get per pixel
#   - For 1 class and background, use n_classes=1
#   - For 2 classes, use n_classes=1
#   - For N > 2 classes, use n_classes=N

logging.info(f'Network:\n'
              f'\t{net.n_channels} input channels\n'
              f'\t{net.n_classes} output channels (classes)\n'
              f'\t{"Bilinear" if net.bilinear else "Transposed conv"} upscaling')


# faster convolutions, but more 
torch.backends.cudnn.benchmark=True
# Train
train_net(net=net,
          epochs=10,
          batch_size=32,
          lr=0.001,
          device=device,
          img_scale=1.0,
          val_percent= 30.0/ 100)


INFO: Using device cuda
INFO: Network:
	6 input channels
	1 output channels (classes)
	Bilinear upscaling
INFO: Starting training:
        Epochs:          10
        Batch size:      32
        Learning rate:   0.001
        Training size:   7000
        Validation size: 3000
        Checkpoints:     True
        Device:          cuda
        Images scaling:  1.0
    
Epoch 1/10: 100%|██████████| 7000/7000 [01:59<00:00, 58.54img/s, loss (batch)=0.987]
INFO: Created checkpoint directory
INFO: Checkpoint 1 saved !
Epoch 2/10: 100%|██████████| 7000/7000 [01:55<00:00, 60.57img/s, loss (batch)=0.989]
INFO: Checkpoint 2 saved !
Epoch 3/10: 100%|██████████| 7000/7000 [01:55<00:00, 60.58img/s, loss (batch)=1.01]
INFO: Checkpoint 3 saved !
Epoch 4/10: 100%|██████████| 7000/7000 [01:55<00:00, 60.58img/s, loss (batch)=0.997]
INFO: Checkpoint 4 saved !
Epoch 5/10: 100%|██████████| 7000/7000 [01:55<00:00, 60.55img/s, loss (batch)=1.01]
INFO: Checkpoint 5 saved !
Epoch 6/10: 100%|██████████| 7000/7

In [9]:
!cp checkpoints/*.pth '/content/drive/My Drive/Untitled folder/'

cp: target '/content/drive/My Drive/Untitled folder/' is not a directory


In [9]:
c ="Dataset/bg/bg32.jpg"
print(c)

Dataset/bg/bg32.jpg


In [0]:
del net

In [0]:
!rm -rf sample50/

In [0]:
# predict
import argparse
import logging
import os

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

from unet import UNet
from utils.data_vis import plot_img_and_mask
from utils.dataset import BasicDataset


def predict_img2(net,
                full_img,
                device,
                scale_factor=1,
                out_threshold=0.5):
    net.eval()

    img = torch.from_numpy(BasicDataset.preprocess(full_img, scale_factor))

    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        depth, mask = net(img) #depth, mask

        if net.n_classes > 1:
            probs1 = F.softmax(depth, dim=1)
        else:
            probs1 = torch.sigmoid(depth)
            ############2
        if net.n_classes > 1:
            probs2 = F.softmax(mask, dim=1)
        else:
            probs2 = torch.sigmoid(mask)

        probs1 = probs1.squeeze(0)

        probs2 = probs2.squeeze(0)

        tf = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.Resize(full_img.size[1]),
                transforms.ToTensor()
            ]
        )

        probs1 = tf(probs1.cpu())
        probs2 = tf(probs2.cpu())
        full_depth = probs1.squeeze().cpu().numpy()
        full_mask = probs2.squeeze().cpu().numpy()
    full_depth = full_depth - np.min(full_depth)
    full_depth = full_depth / np.max(full_depth)
    return full_depth , full_mask #> out_threshold


def get_output_filenames(args):
    in_files = args.input
    out_files = []

    if not args.output:
        for f in in_files:
            pathsplit = os.path.splitext(f)
            out_files.append("{}_OUT{}".format(pathsplit[0], pathsplit[1]))
    elif len(in_files) != len(args.output):
        logging.error("Input files and output files are not of the same length")
        raise SystemExit()
    else:
        out_files = args.output

    return out_files


def mask_to_image(mask):
    return Image.fromarray((mask * 255).astype(np.uint8))



In [0]:
!mkdir sample50
!mkdir sample50/out
!mkdir sample50/ind
!mkdir sample50/act
from predict import *
from PIL import Image
sdir = 'sample50/out/'
actdir ='sample50/act/'
indir ='sample50/ind/'
for i,idx in enumerate(os.listdir('data/fgbg/')):
  if i==10:
    break;
  img = Image.open('data/fgbg/'+idx)
  img.save(indir+idx)
  act = Image.open('data/depth/'+idx)
  act.save(actdir+idx)
  depth, mask = predict_img2(net=net, full_img=img, scale_factor=1.0, out_threshold=0.1, device=device)
  result1 = mask_to_image(mask)
  result2 = mask_to_image(depth)
  result1.save(sdir+idx)
  result2.save(sdir+'depth'+idx)


In [0]:
mkdir sample50/out

In [0]:
!cp -r sample50 '/content/drive/My Drive/'

In [0]:
# dataset
from os.path import splitext
from os import listdir
import numpy as np
from glob import glob
import torch
from torch.utils.data import Dataset
import logging
from PIL import Image


class BasicDataset(Dataset):
    def __init__(self, root):
        self.root = root
        data_file = open(root+'Dataset/'+'label_data.csv')
        self.data = data_file.readlines()

    def __len__(self):
        return len(self.data)

    @classmethod
    def preprocess(cls, pil_img):

        img_nd = np.array(pil_img)

        if len(img_nd.shape) == 2:
            img_nd = np.expand_dims(img_nd, axis=2)

        # HWC to CHW
        img_trans = img_nd.transpose((2, 0, 1))
        if img_trans.max() > 1:
            img_trans = img_trans / 255
        return img_trans

    def __getitem__(self, i):
        idx = self.data[i].split(';')
        bg = Image.open(self.root + idx[0].replace('"',''))
        fgbg = Image.open(self.root + idx[1].replace('"',''))
        mask = Image.open(self.root + idx[2].replace('"',''))
        depth = Image.open(self.root + idx[3].replace('"','').replace('\n',''))
        bg = self.preprocess(bg)
        fgbg = self.preprocess(fgbg)
        mask = self.preprocess(mask)
        depth =self.preprocess(depth)
        return {'bg' : torch.from_numpy(bg), 'fgbg': torch.from_numpy(fgbg), 'mask': torch.from_numpy(mask), 'depth': torch.from_numpy(depth)}


In [0]:
""" Full assembly of the parts to form the complete network """
#unet_model
import torch.nn.functional as F

from .unet_parts import *


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 32)
        self.down1 = Down(32, 64)
        self.down2 = Down(64, 128)
        self.down3 = Down(128, 256)
        factor = 2 if bilinear else 1
        self.down4 = Down(256, 512 // factor)
        self.up1 = Up(512, 256 // factor, bilinear)
        self.up2 = Up(256, 128 // factor, bilinear)
        self.up3 = Up(128, 64 // factor, bilinear)
        self.up4 = Up(64, 32, bilinear)
        self.outc = OutConv(32, n_classes)

    def forward(self, data):
        dat1 = data['fgbg']
        dat2 = data['bg']
        x = torch.cat([dat1,dat2], dim = 1)
        y = x
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        rt_depth = self.outc(x)

        y1 = self.inc(y) #------------
        y2 = self.down1(y1) #------------
        y3 = self.down2(y2) #------------ 
        y4 = self.down3(y3) #------------
        y5 = self.down4(y4) #------------
        y = self.up1(y5, y4) #------------
        y = self.up2(y, y3) #------------
        y = self.up3(y, y2) #------------
        y = self.up4(y, y1) #------------
        rt_mask = self.outc(y) #------------
        return rt_depth, rt_mask #


In [0]:
# predict
import argparse
import logging
import os

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

from unet import UNet
from utils.data_vis import plot_img_and_mask
from utils.dataset import BasicDataset


def predict_img(net,
                full_img,
                device,
                scale_factor=1,
                out_threshold=0.5):
    net.eval()

    img = torch.from_numpy(BasicDataset.preprocess(full_img))

    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        depth, mask = net(img) #depth, mask

        if net.n_classes > 1:
            probs1 = F.softmax(depth, dim=1)
        else:
            probs1 = torch.sigmoid(depth)
            ############2
        if net.n_classes > 1:
            probs2 = F.softmax(mask, dim=1)
        else:
            probs2 = torch.sigmoid(mask)

        probs1 = probs1.squeeze(0)

        probs2 = probs2.squeeze(0)

        tf = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.Resize(full_img.size[1]),
                transforms.ToTensor()
            ]
        )

        probs1 = tf(probs1.cpu())
        probs2 = tf(probs2.cpu())
        full_depth = probs1.squeeze().cpu().numpy()
        full_mask = probs2.squeeze().cpu().numpy()
    full_depth = full_depth - np.min(full_depth)
    full_depth = full_depth / np.max(full_depth)
    return full_depth , full_mask# > out_threshold


def get_output_filenames(args):
    in_files = args.input
    out_files = []

    if not args.output:
        for f in in_files:
            pathsplit = os.path.splitext(f)
            out_files.append("{}_OUT{}".format(pathsplit[0], pathsplit[1]))
    elif len(in_files) != len(args.output):
        logging.error("Input files and output files are not of the same length")
        raise SystemExit()
    else:
        out_files = args.output

    return out_files


def mask_to_image(mask):
    return Image.fromarray((mask * 255).astype(np.uint8))

